In [ ]:
install.packages("mlflow")
install.packages("carrier")

In [ ]:
# Load required libraries
library(sparklyr)
library(dplyr)
library(mlflow)
library(rpart)

In [ ]:
# 1. Read the iris_data table from Unity Catalog into a dataframe

# Read the iris data from the specified table
iris_tbl <- tbl(spark, "pedroz_e2edata_dev.default.iris_features")

# Collect data into R for modeling
iris_data <- collect(iris_tbl)

In [ ]:
# 2. Load the model from Unity Catalog using mlflow_load_model
model <- mlflow_load_model("models:/pedroz_e2edata_dev.default.iris_r_class_model_uc@champion")

In [ ]:
model

In [ ]:
# model is the crate you loaded
preds <- model(iris_data)


In [ ]:
# Add predictions to the iris_data dataframe
iris_data$prediction <- preds

# Copy iris_data with predictions back to Spark
iris_pred_tbl <- copy_to(spark, iris_data, "iris_data_with_preds", overwrite = TRUE)

In [ ]:
display(iris_pred_tbl)

In [ ]:
# Save the Spark DataFrame to a Unity Catalog table using sparklyr
spark_write_table(
  iris_pred_tbl,
  name = "pedroz_e2edata_dev.default.iris_data_with_preds",
  mode = "overwrite"
)